In [1]:
###########################################################################
#  Code for computing the RPS and IR scores for a given evaluation period
###########################################################################

#For simplicity, in this example it is assumed that the data provided cover a single evaluation period.
#This period is specified through the min/max date of the asset prices data set.
#If you wish to compute RPS/IR for multiple periods, you'll have to execute 
#the script multiple times, each time using a different, appropriate input.

import pandas as pd
import numpy as np
from statistics import stdev

In [2]:
CALC_START_DATE = "2022/03/04" #last known EOD price to calculate, date format YYYYMMDD

In [3]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [4]:
data_in     = "../DataRaw/"
data_folder = "../DataWork/"

In [5]:
#Read asset prices data (as provided by the M6 submission platform)
asset_data = pd.read_csv(data_in + "assets_m6.csv")

#Read submission file (similar to the template provided by the M6 submission platform)
submission_data = pd.read_csv(data_folder + "submit_2022_03_05.csv")

hist_data = asset_data
submission = submission_data

In [6]:
hist_data

,symbol,date,price
0,ABBV,2022/01/31,136.890
1,ACN,2022/01/31,353.580
2,AEP,2022/01/31,89.618
3,AIZ,2022/01/31,151.876
4,ALLE,2022/01/31,122.730
...,...,...,...
3405,XLP,2022/03/18,74.190
3406,XLU,2022/03/18,70.810
3407,XLV,2022/03/18,136.990
3408,XLY,2022/03/18,181.880


In [7]:
hist_data = hist_data[hist_data.date >= CALC_START_DATE]
hist_data

,symbol,date,price
2310,ABBV,2022/03/04,150.56
2311,ACN,2022/03/04,314.16
2312,AEP,2022/03/04,96.33
2313,AIZ,2022/03/04,169.39
2314,ALLE,2022/03/04,117.29
...,...,...,...
3405,XLP,2022/03/18,74.19
3406,XLU,2022/03/18,70.81
3407,XLV,2022/03/18,136.99
3408,XLY,2022/03/18,181.88


In [8]:
submission

,ID,Rank1,Rank2,Rank3,Rank4,Rank5,Decision
0,ABBV,0,1,0,0,0,0.00
1,ACN,0,0,0,1,0,0.01
2,AEP,0,0,0,1,0,0.01
3,AIZ,0,0,0,1,0,0.01
4,ALLE,0,0,0,0,1,0.02
5,AMAT,0,0,0,0,1,0.02
6,AMP,0,0,1,0,0,0.01
7,AMZN,0,0,0,0,1,0.02
8,AVB,0,0,0,1,0,0.01
9,AVY,1,0,0,0,0,0.00


In [9]:
#Function for computing RPS
def RPS_calculation(hist_data, submission, asset_no=100):
    
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                              'symbol' : asset,
                                              'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price  = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0],
                                  'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    #Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:
        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : ranking}
    
    return(output)
   
#Function for computing IR
def IR_calculation(hist_data, submission):
    
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                              'symbol' : asset,
                                              'price' : right_price}, ignore_index=True)

    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])

    #Investment weights
    weights = submission[["ID","Decision"]]

    RET = pd.DataFrame()

    for i in range(len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        temp = temp.sort_values(by='date')
        temp.reset_index(inplace=True, drop=True)
        RET = RET.append(temp.price.pct_change()*weights.loc[weights.ID==asset_id[i]].Decision.values[0])

    ret = np.log(1+RET.sum()[1:])
    sum_ret = sum(ret)
    sdp = stdev(ret)
    
    output = {'IR' : sum_ret/sdp,
              'details' : RET} #list(ret)}
    
    return output

In [10]:
#Run evaluation
rps_dict = RPS_calculation(hist_data = hist_data , submission = submission_data)
rps = rps_dict['RPS']
ir_dict = IR_calculation(hist_data, submission)
ir = ir_dict['IR']

print(f'RPS {rps:.3f} IR {ir:.3f}')

RPS 0.320 IR 2.144


In [11]:
rps_dict['details'].sort_values(by='ID').reset_index(drop=True)

,ID,Return,Position,Rank,Rank1,Rank2,Rank3,Rank4,Rank5
0,ABBV,0.057386,75.0,4.0,0.0,0.0,0.0,1.0,0.0
1,ACN,0.031162,52.0,3.0,0.0,0.0,1.0,0.0,0.0
2,AEP,-0.023461,13.0,1.0,1.0,0.0,0.0,0.0,0.0
3,AIZ,0.041502,60.0,3.0,0.0,0.0,1.0,0.0,0.0
4,ALLE,0.007247,31.0,2.0,0.0,1.0,0.0,0.0,0.0
5,AMAT,0.074757,81.0,5.0,0.0,0.0,0.0,0.0,1.0
6,AMP,0.068670,80.0,4.0,0.0,0.0,0.0,1.0,0.0
7,AMZN,0.107178,96.0,5.0,0.0,0.0,0.0,0.0,1.0
8,AVB,-0.007924,20.0,1.0,1.0,0.0,0.0,0.0,0.0
9,AVY,0.021063,42.0,3.0,0.0,0.0,1.0,0.0,0.0


In [12]:
submission.sort_values(by='ID')

,ID,Rank1,Rank2,Rank3,Rank4,Rank5,Decision,RPS
0,ABBV,0,1,0,0,0,0.00,0.4
1,ACN,0,0,0,1,0,0.01,0.2
2,AEP,0,0,0,1,0,0.01,0.6
3,AIZ,0,0,0,1,0,0.01,0.2
4,ALLE,0,0,0,0,1,0.02,0.6
5,AMAT,0,0,0,0,1,0.02,0.0
6,AMP,0,0,1,0,0,0.01,0.2
7,AMZN,0,0,0,0,1,0.02,0.0
8,AVB,0,0,0,1,0,0.01,0.6
9,AVY,1,0,0,0,0,0.00,0.4
